In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stat
import statsmodels.api as sm
import statsmodels.formula.api as smf#, ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import ttest_ind, f_oneway
import scipy.stats as stats
import sys
from pathlib import Path
# # Add src directory to the Python path
sys.path.append(str(Path.cwd().parent))
from data_analysis.df_dataattribute_analysis import categorize_columns
from data_sourcing.import_export_data import get_data, save_data
from data_analysis.df_ttest_anova_analysis import run_one_sample_ttest, run_univariate_tests
from data_analysis.df_multicollinearity_analysis import compute_vif
from src.data_analysis.df_correlation_analysis import compute_correlations

In [2]:
df = get_data('featured_engineered_testing.csv', 'processed_data')
df.head(5)

2025-01-26 17:16:27,764 - INFO - Loading data from: ..\data\processed\featured_engineered_testing.csv


,ATL_OR_DR,CAMPAIGN_TYPE,CHANNEL,COST,FREE_TRIALS,HOLIDAY_FLAG,MONTH,ATL_OR_DR_CAMPAIGN_TYPE,ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL,ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH,...,LOG_COST,LOG_FREE_TRIALS,LOG_COST_PER_FREE_TRIALS,LOG_COST_BY_FREE_TRIALS,AVG_COST_MONTH,MEDIAN_COST_MONTH,AVG_COST_CAMPAIGN_TYPE_CHANNEL,MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL,AVG_FREE_TRIALS_MONTH,MEDIAN_FREE_TRIALS_MONTH
0,DR - Direct Response,Title,paid social,7784.31,86401.15,False,June,DR - Direct Response_Title,DR - Direct Response_Title_paid social,DR - Direct Response_Title_paid social_June,...,8.959994,11.366768,0.086265,20.326622,16887.563506,3643.265,14781.334567,3309.50,42865.334723,25054.485
1,DR - Direct Response,Title,app,2474.31,2956.74,False,August,DR - Direct Response_Title,DR - Direct Response_Title_app,DR - Direct Response_Title_app_August,...,7.814121,7.992181,0.608045,15.805560,14172.931604,2955.060,15454.451540,3477.08,43953.768382,28127.390
2,ATL - Above The Line,Title,paid social,10222.82,12513.08,False,July,ATL - Above The Line_Title,ATL - Above The Line_Title_paid social,ATL - Above The Line_Title_paid social_July,...,9.232476,9.434610,0.597171,18.666908,13667.902515,2994.470,14781.334567,3309.50,45959.687637,27091.210
3,DR - Direct Response,Title,app,49631.87,17207.04,True,December,DR - Direct Response_Title,DR - Direct Response_Title_app,DR - Direct Response_Title_app_December,...,10.812409,9.753132,1.356967,20.565462,18937.983302,3240.025,15454.451540,3477.08,47495.480279,28640.060
4,ATL - Above The Line,Title,paid social,2081.06,21758.33,False,August,ATL - Above The Line_Title,ATL - Above The Line_Title_paid social,ATL - Above The Line_Title_paid social_August,...,7.641113,9.987798,0.091343,17.628385,14172.931604,2955.060,14781.334567,3309.50,43953.768382,28127.390


### Multi-collinearity Analysis

In [ ]:
vif_result = compute_vif(
    data=df,
)
vif_result

Categorical columns detected: ['ATL_OR_DR', 'CAMPAIGN_TYPE', 'CHANNEL', 'HOLIDAY_FLAG', 'MONTH', 'ATL_OR_DR_CAMPAIGN_TYPE', 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL', 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH', 'ATL_OR_DR_CAMPAIGN_TYPE_MONTH', 'ATL_OR_DR_CHANNEL', 'ATL_OR_DR_CHANNEL_MONTH', 'ATL_OR_DR_MONTH', 'CAMPAIGN_TYPE_CHANNEL', 'CAMPAIGN_TYPE_CHANNEL_MONTH', 'CAMPAIGN_TYPE_MONTH', 'CHANNEL_MONTH']
DataFrame after one-hot encoding:
DataFrame after converting all columns to numeric:
COST                               float64
FREE_TRIALS                        float64
COST_PER_FREE_TRIALS               float64
COST_BY_FREE_TRIALS                float64
LOG_COST                           float64
                                    ...   
CHANNEL_MONTH_youtube_March           bool
CHANNEL_MONTH_youtube_May             bool
CHANNEL_MONTH_youtube_November        bool
CHANNEL_MONTH_youtube_October         bool
CHANNEL_MONTH_youtube_September       bool
Length: 1475, dtype: object
Checking for NaN o

In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif_test(df):
    """
    Calculate Variance Inflation Factor (VIF) for a DataFrame with mixed data types.

    Parameters:
        df (pd.DataFrame): Input DataFrame with both categorical and continuous features.

    Returns:
        pd.DataFrame: DataFrame containing features and their VIF values.
    """
    # Separate categorical and continuous columns
    categorical_cols = df.select_dtypes(include=['object', 'category', 'bool']).columns
    continuous_cols = df.select_dtypes(include=['number']).columns

    # One-hot encode categorical variables
    if len(categorical_cols) > 0:
        encoder = OneHotEncoder(drop='first', sparse=False)
        encoded_cats = pd.DataFrame(encoder.fit_transform(df[categorical_cols]),
                                    columns=encoder.get_feature_names_out(categorical_cols),
                                    index=df.index)
    else:
        encoded_cats = pd.DataFrame(index=df.index)  # Empty DataFrame if no categorical columns

    # Standardize continuous variables
    if len(continuous_cols) > 0:
        scaler = StandardScaler()
        scaled_cont = pd.DataFrame(scaler.fit_transform(df[continuous_cols]),
                                   columns=continuous_cols,
                                   index=df.index)
    else:
        scaled_cont = pd.DataFrame(index=df.index)  # Empty DataFrame if no continuous columns

    # Combine processed features
    processed_df = pd.concat([encoded_cats, scaled_cont], axis=1)

    # Calculate VIF
    vif_data = pd.DataFrame()
    vif_data["Feature"] = processed_df.columns
    vif_data["VIF"] = [variance_inflation_factor(processed_df.values, i)
                       for i in range(processed_df.shape[1])]

    return vif_data


In [ ]:
# # Example DataFrame
# data = {
#     'feature_1': [1.0, 2.0, 3.0, 4.0],
#     'feature_2': [5.0, 6.0, 7.0, 8.0],
#     'category': ['A', 'B', 'A', 'B'],
#     'flag': [True, False, True, False]
# }
# df = pd.DataFrame(data)

# Run the VIF function
vif_result = calculate_vif_test(df)

# Display the VIF results
print(vif_result)


c:\Users\cheng\Workspace\Paramount\.venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### One Sample T-Test Aanalysis

In [3]:

categorical = ["ATL_OR_DR", "HOLIDAY_FLAG"]
quantitative = ["FREE_TRIALS", "LOG_FREE_TRIALS"]


# 2) Using a dictionary for different hypothesized means per column:
results_2 = run_one_sample_ttest(
    df=df,
    categorical_cols=categorical,
    quantitative_cols=quantitative,
    popmean={"FREE_TRIALS": 277769, "LOG_FREE_TRIALS": 10.22},  # different popmeans
    stat_type="median",
    alpha=0.05,  # more stringent significance level
    sort_results=True
)
results_2

,Categorical Variable,Category,Quantitative Variable,Sample Size,Sample Stat,T-statistic,p-value,Decision,Error
0,ATL_OR_DR,DR - Direct Response,FREE_TRIALS,5507,27833.420000,-294.961263,0.000000,Reject H0,None
1,ATL_OR_DR,ATL - Above The Line,FREE_TRIALS,8601,27769.030000,-349.593644,0.000000,Reject H0,None
2,HOLIDAY_FLAG,False,FREE_TRIALS,12629,27775.900000,-430.288018,0.000000,Reject H0,None
3,HOLIDAY_FLAG,True,FREE_TRIALS,1479,27789.190000,-153.775053,0.000000,Reject H0,None
4,HOLIDAY_FLAG,False,LOG_FREE_TRIALS,12629,10.231960,1.154430,0.248346,Fail to Reject H0,None
5,ATL_OR_DR,ATL - Above The Line,LOG_FREE_TRIALS,8601,10.231713,0.750651,0.452883,Fail to Reject H0,None
6,ATL_OR_DR,DR - Direct Response,LOG_FREE_TRIALS,5507,10.234029,0.586744,0.557399,Fail to Reject H0,None
7,HOLIDAY_FLAG,True,LOG_FREE_TRIALS,1479,10.232438,-0.417617,0.676288,Fail to Reject H0,None


### ANOVA Testing Analysis

In [4]:
hashmap = categorize_columns(df)
categorical = hashmap['categorical']
categorical.remove('ATL_OR_DR')
categorical.remove('HOLIDAY_FLAG')
categorical

['CAMPAIGN_TYPE',
 'CHANNEL',
 'MONTH',
 'ATL_OR_DR_CAMPAIGN_TYPE',
 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL',
 'ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH',
 'ATL_OR_DR_CAMPAIGN_TYPE_MONTH',
 'ATL_OR_DR_CHANNEL',
 'ATL_OR_DR_CHANNEL_MONTH',
 'ATL_OR_DR_MONTH',
 'CAMPAIGN_TYPE_CHANNEL',
 'CAMPAIGN_TYPE_CHANNEL_MONTH',
 'CAMPAIGN_TYPE_MONTH',
 'CHANNEL_MONTH']

In [5]:
quantitative = ["FREE_TRIALS", "LOG_FREE_TRIALS"]

anova_results_df = run_univariate_tests(
    df=df,
    categorical_cols=categorical,
    quantitative_cols=quantitative,
    test_type="anova"
)
anova_results_df

kruskal_results_df = run_univariate_tests(
    df=df,
    categorical_cols=categorical,
    quantitative_cols=quantitative,
    test_type="kruskal"
)


In [6]:
kruskal_results_df = run_univariate_tests(
    df=df,
    categorical_cols=categorical,
    quantitative_cols=quantitative,
    test_type="kruskal"
)
kruskal_results_df

,Categorical Variable,Quantitative Variable,H-statistic,p-value,Error
0,MONTH,FREE_TRIALS,15.421565,0.163999,None
1,MONTH,LOG_FREE_TRIALS,15.421565,0.163999,None
2,ATL_OR_DR_MONTH,LOG_FREE_TRIALS,26.406948,0.282034,None
3,ATL_OR_DR_MONTH,FREE_TRIALS,26.406948,0.282034,None
4,CHANNEL_MONTH,LOG_FREE_TRIALS,193.675803,0.493063,None
5,CHANNEL_MONTH,FREE_TRIALS,193.675803,0.493063,None
6,ATL_OR_DR_CHANNEL_MONTH,LOG_FREE_TRIALS,233.572700,0.586903,None
7,ATL_OR_DR_CHANNEL_MONTH,FREE_TRIALS,233.572700,0.586903,None
8,CAMPAIGN_TYPE_CHANNEL,LOG_FREE_TRIALS,38.981914,0.604215,None
9,CAMPAIGN_TYPE_CHANNEL,FREE_TRIALS,38.981914,0.604215,None


### Pearson/Spearmans Correlation Testing

In [8]:

# 1) Pearson correlation (numeric only)
pearson_results = compute_correlations(
    df=df,
    include_categorical=False,
    cat_cols=None  # Do not encode categorical variables
)
#print("Pearson Correlation Results:\n", pearson_results)

# 2) Spearman correlation (with categorical one-hot encoding)
spearman_results = compute_correlations(
    df=df,
    include_categorical=True,  # Encode categorical variables
    drop_first=True,
    cat_cols=categorical
)
#print("\nSpearman Correlation Results (with one-hot encoding):\n", spearman_results)


In [9]:
pearson_results[pearson_results['p-value'] < 0.05]

,Variable 1,Variable 2,Correlation,p-value,Method,Error
0,AVG_FREE_TRIALS_MONTH,MEDIAN_FREE_TRIALS_MONTH,0.670824,0.000000,pearson,None
1,COST_PER_FREE_TRIALS,LOG_COST_PER_FREE_TRIALS,0.518058,0.000000,pearson,None
2,COST_BY_FREE_TRIALS,LOG_COST_BY_FREE_TRIALS,0.327406,0.000000,pearson,None
3,AVG_COST_CAMPAIGN_TYPE_CHANNEL,MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL,0.354669,0.000000,pearson,None
4,FREE_TRIALS,LOG_COST_BY_FREE_TRIALS,0.347763,0.000000,pearson,None
...,...,...,...,...,...,...
62,LOG_COST_PER_FREE_TRIALS,MEDIAN_COST_MONTH,0.018579,0.027329,pearson,None
63,MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL,AVG_FREE_TRIALS_MONTH,-0.017045,0.042912,pearson,None
64,COST,MEDIAN_FREE_TRIALS_MONTH,0.017023,0.043191,pearson,None
65,COST_PER_FREE_TRIALS,AVG_COST_MONTH,0.016962,0.043942,pearson,None


In [10]:
spearman_results

,Variable 1,Variable 2,Correlation,p-value,Method,Error
0,AVG_FREE_TRIALS_MONTH,MEDIAN_FREE_TRIALS_MONTH,0.567204,0.000000,spearman,None
1,COST_PER_FREE_TRIALS,LOG_COST,0.865113,0.000000,spearman,None
2,COST_PER_FREE_TRIALS,COST_BY_FREE_TRIALS,0.515490,0.000000,spearman,None
3,LOG_COST,LOG_COST_BY_FREE_TRIALS,0.865524,0.000000,spearman,None
4,COST_PER_FREE_TRIALS,LOG_COST_BY_FREE_TRIALS,0.515490,0.000000,spearman,None
...,...,...,...,...,...,...
86,LOG_FREE_TRIALS,MEDIAN_COST_MONTH,0.001985,0.813664,spearman,None
87,FREE_TRIALS,MEDIAN_COST_MONTH,0.001985,0.813664,spearman,None
88,FREE_TRIALS,AVG_COST_CAMPAIGN_TYPE_CHANNEL,0.000420,0.960183,spearman,None
89,LOG_FREE_TRIALS,AVG_COST_CAMPAIGN_TYPE_CHANNEL,0.000420,0.960183,spearman,None


In [11]:
# import pandas as pd
# import statsmodels.api as sm
# from statsmodels.formula.api import ols

# # Example: Assuming your data is in a DataFrame called df
# # Replace 'FREE_TRIALS' with your dependent variable
# # Replace other variable names with the actual column names from your dataset

# # Create the OLS formula
# formula = """
# FREE_TRIALS ~ HOLIDAY_FLAG
# + ATL_OR_DR + CAMPAIGN_TYPE + CHANNEL + MONTH
# + ATL_OR_DR_CAMPAIGN_TYPE + ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL
# + ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL_MONTH + ATL_OR_DR_CAMPAIGN_TYPE_MONTH
# + ATL_OR_DR_CHANNEL + ATL_OR_DR_CHANNEL_MONTH + ATL_OR_DR_MONTH
# + CAMPAIGN_TYPE_CHANNEL + CAMPAIGN_TYPE_CHANNEL_MONTH + CAMPAIGN_TYPE_MONTH + CHANNEL_MONTH
# + COST + COST_PER_FREE_TRIALS + COST_BY_FREE_TRIALS
# + LOG_COST + LOG_COST_PER_FREE_TRIALS + LOG_COST_BY_FREE_TRIALS
# + AVG_COST_MONTH + MEDIAN_COST_MONTH
# + AVG_COST_CAMPAIGN_TYPE_CHANNEL + MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL
# + AVG_FREE_TRIALS_MONTH + MEDIAN_FREE_TRIALS_MONTH
# """
# # formula = """
# # FREE_TRIALS ~ HOLIDAY_FLAG
# # + MONTH + CHANNEL + CAMPAIGN_TYPE + ATL_OR_DR
# # + COST + COST_PER_FREE_TRIALS + LOG_COST
# # + MONTH:CHANNEL + CAMPAIGN_TYPE:ATL_OR_DR
# # """
# #formula = "FREE_TRIALS ~ MONTH * ATL_OR_DR + CHANNEL"

# # Fit the OLS model
# # model = ols(formula, data=df).fit()

# import statsmodels.api as sm
# model = sm.GLM(df['FREE_TRIALS'], sm.add_constant(X), family=sm.families.Poisson()).fit()
# print(model.summary())


# # Print the summary of the regression model
# print(model.summary())


In [12]:
# pd.DataFrame(model.summary())

In [13]:
# # # Define a function to plot boxplots and detect outliers based on IQR
# # def plot_outliers(column, ax):
# #     ax.boxplot(data[column], vert=False, patch_artist=True, boxprops=dict(facecolor="lightblue"))
# #     ax.set_title(f'Boxplot of {column}')
# #     ax.set_xlabel(column)

# # # Create boxplots for COST and FREE_TRIALS
# # fig, axes = plt.subplots(1, 2, figsize=(12, 5))
# # plot_outliers('COST', axes[0])
# # plot_outliers('FREE_TRIALS', axes[1])
# # plt.tight_layout()
# # plt.show()

# # Calculate outlier thresholds using IQR
# def calculate_outliers(column):
#     q1 = data[column].quantile(0.25)
#     q3 = data[column].quantile(0.75)
#     iqr = q3 - q1
#     lower_bound = q1 - 1.5 * iqr
#     upper_bound = q3 + 1.5 * iqr
#     outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
#     return lower_bound, upper_bound, outliers

# # Remove outliers based on IQR thresholds for LOG_COST and LOG_FREE_TRIALS
# log_cost_bounds = calculate_outliers('LOG_COST')
# log_free_trials_bounds = calculate_outliers('LOG_FREE_TRIALS')

# # Filter data to exclude outliers in LOG_COST and LOG_FREE_TRIALS
# filtered_log_data = data[
#     (data['LOG_COST'] >= log_cost_bounds[0]) & (data['LOG_COST'] <= log_cost_bounds[1]) &
#     (data['LOG_FREE_TRIALS'] >= log_free_trials_bounds[0]) & (data['LOG_FREE_TRIALS'] <= log_free_trials_bounds[1])
# ]

# # Display the size of the dataset before and after outlier removal based on log variables
# log_filtered_size = filtered_log_data.shape[0]

# log_filtered_size


In [14]:
# from sklearn.preprocessing import StandardScaler

# # Select numerical columns to standardize
# numerical_features = ['LOG_COST', 'LOG_COST_SQ']
# scaler = StandardScaler()

# # Standardize numerical features
# filtered_log_data_encoded[numerical_features] = scaler.fit_transform(
#     filtered_log_data_encoded[numerical_features]
# )

# # Re-run OLS regression with standardized features
# X_standardized = filtered_log_data_encoded[['LOG_COST', 'LOG_COST_SQ'] +
#                                            [col for col in filtered_log_data_encoded.columns
#                                             if col.startswith(('ATL_OR_DR_', 'CAMPAIGN_TYPE_', 'CHANNEL_'))]]
# X_standardized = sm.add_constant(X_standardized)  # Add constant for OLS

# y_standardized = filtered_log_data_encoded['LOG_FREE_TRIALS']

# # Fit OLS regression with standardized features
# ols_standardized_model = sm.OLS(y_standardized, X_standardized).fit()

# # Display the updated summary
# ols_standardized_summary = ols_standardized_model.summary()
# ols_standardized_summary